In [6]:
# Gaussian Process Model 
# ref) https://pyro.ai/examples/gp.html

import os
import io 
import time
import numpy as np
import torch

from scipy.stats import norm
import pyro
import pyro.contrib.gp as gp


y = torch.Tensor(np.array([
    0.0, 0.1, 0.2, 0.3, 0.2, 0.1, -0.2, -0.3, -0.2, 0.1, -0.1, 0.0
]))
X = y[:-1]
y = y[1:]

gpm_lookback_win = 10
X = X[-gpm_lookback_win:]
y = y[-gpm_lookback_win:]


gpm_kernel = gp.kernels.Matern32
gpm_noise = 0.0

gpm = gp.models.GPRegression(X, y, gpm_kernel, 
                                    noise=torch.tensor(gpm_noise), mean_function=None, jitter=1e-6)
gpm.set_data(X, y)
optimizer = torch.optim.Adam(gpm.parameters(), lr=0.005)
loss_fn = pyro.infer.Trace_ELBO().differentiable_loss

# optim_tracker = OptimTracker(use_early_stop=True, patience=configs.max_gp_opt_patience, verbose=False, save_to_file=False)
num_batch = 10
max_gp_opt_steps = 2000
for step in range(1, max_gp_opt_steps, num_batch):

    loss = gp.util.train(gpm, optimizer, loss_fn, num_steps=num_batch)
    mean_loss = np.mean(loss).item() / len(y) # mean loss for one-step
    print(f"after training {step} times, loss={mean_loss:.4f}")
    # optim_tracker(mean_loss, None)
    # if optim_tracker.early_stop:
    #     break
# logger.debug(f"Adj.HPO: when lb_win_size={gpm_lookback_win}, after training {step} times, loss={mean_loss:.4f}")
print(f"Adj.HPO: when lb_win_size={gpm_lookback_win}, after training {step} times, loss={mean_loss:.4f}")


TypeError: only integer tensors of a single element can be converted to an index